## Initial parameters

In [1]:
seed = 42
dataset = 'adult'
folder_name = 'results/' + dataset + '/non_private'

## Function

In [2]:
def write(folder_name, values):
    with open(folder_name + "/LGBM_BO_results.csv", mode='a', newline='') as scores_file:
        scores_writer = csv.writer(scores_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        scores_writer.writerow(values)
    scores_file.close()
    
def fairness_metrics(df_fm, protected_attribute):
    
    fair_met = {# Statistical Parity
                "SP_a_1": np.nan,
                "SP_a_0": np.nan,
                "DI": np.nan, # Disparate Impact
                "SPD": np.nan, # Statistical Parity Difference
                # Equal Opportunity
                "EO_a_1": np.nan,
                "EO_a_0": np.nan,
                "EOD": np.nan, # Equal Opportunity Difference
                # Overall Accuracy
                "OA_a_1": np.nan,
                "OA_a_0": np.nan,
                "OAD": np.nan, # Overall Accuracy Difference
                }
    
    # Filtering datasets for fairness metrics
    df_a_1 = df_fm.loc[df_fm[protected_attribute+"_1"]==1]
    df_a_0 = df_fm.loc[df_fm[protected_attribute+"_1"]==0]

    # Calculate Statistical Parity per group
    SP_a_1 = df_a_1.loc[df_a_1["y_pred"]==1].shape[0] / df_a_1.shape[0]
    SP_a_0 = df_a_0.loc[df_a_0["y_pred"]==1].shape[0] / df_a_0.shape[0]
    fair_met["SP_a_1"] = SP_a_1
    fair_met["SP_a_0"] = SP_a_0

    # Disparate Impact
    DI = SP_a_0 / SP_a_1
    fair_met["DI"] = DI
    
    # Statistical Parity Difference
    SPD = SP_a_1 - SP_a_0
    fair_met["SPD"] = SPD

    # Equal Opportunity
    EO_a_1 = recall_score(df_a_1[target], df_a_1['y_pred'])
    EO_a_0 = recall_score(df_a_0[target], df_a_0['y_pred'])
    fair_met["EO_a_1"] = EO_a_1
    fair_met["EO_a_0"] = EO_a_0

    # Equal Opportunity Difference
    EOD = EO_a_1 - EO_a_0
    fair_met["EOD"] = EOD

    # Overall Accuracy
    OA_a_1 = accuracy_score(df_a_1[target], df_a_1['y_pred'])
    OA_a_0 = accuracy_score(df_a_0[target], df_a_0['y_pred'])
    fair_met["OA_a_1"] = OA_a_1
    fair_met["OA_a_0"] = OA_a_0

    # Accuracy per Group Difference
    OAD = OA_a_1 - OA_a_0
    fair_met["OAD"] = OAD
    
    return fair_met    

## Importing

In [3]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
import csv

# sklearn imports
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score, recall_score

# hyper-params opti
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

## Reading dataset

In [4]:
df = pd.read_csv('datasets/db_adult_processed_26k.csv')
df

,age,workclass,education,marital-status,occupation,native-country,relationship,hours-per-week,gender,race,income
0,23,2,9,2,12,38,5,19,0,4,1
1,4,2,15,0,2,38,3,39,1,4,0
2,0,2,1,4,7,38,3,9,1,4,0
3,34,2,11,2,11,0,0,49,1,1,1
4,9,2,9,4,3,38,1,37,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
45844,18,2,9,2,4,40,0,64,1,4,1
45845,20,4,9,2,11,39,0,75,1,1,1
45846,7,2,8,4,11,38,1,54,1,4,0
45847,7,2,15,4,0,38,1,39,0,4,0


## Static parameters

In [5]:
target = 'income'
protected_attribute = 'gender'
test_size = 0.2

## Encoding

In [6]:
lst_df = []
for col in df.columns[:-1]:
    
    lst_col_name = [col+"_{}".format(val) for val in range(len(set(df[col])))]
    
    k = len(set(df[col]))
    
    OHE = np.eye(k)
    
    df_ohe = pd.DataFrame([OHE[val] for val in df[col]], columns=lst_col_name)
    lst_df.append(df_ohe)
df = pd.concat([pd.concat(lst_df, axis=1), df[target]], axis=1)
df

,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,...,hours-per-week_94,hours-per-week_95,gender_0,gender_1,race_0,race_1,race_2,race_3,race_4,income
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
45845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
45846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
45847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0


## Splitting train and test sets

In [7]:
X = copy.deepcopy(df.drop(target, axis=1))
y = copy.deepcopy(df[target])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, stratify=y, random_state=seed)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((36679, 268), (36679,), (9170, 268), (9170,))

## Non-private LGBM

In [8]:
model = LGBMClassifier(random_state=seed, n_jobs=-1, objective="binary")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# performance metrics
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("ACC:", acc)
print("CM:", cm)
print("f1:", f1)
print("AUC:", auc)


df_fm = pd.concat([X_test, y_test], axis=1)
df_fm['y_pred'] = y_pred

print('\nFairness Metrics')

# datasets for fairness metrics
df_a_1 = df_fm.loc[df_fm[protected_attribute+"_1"]==1]
df_a_0 = df_fm.loc[df_fm[protected_attribute+"_1"]==0]

# Statistical Parity
SP_a_1 = df_a_1.loc[df_a_1["y_pred"]==1].shape[0] / df_a_1.shape[0]
SP_a_0 = df_a_0.loc[df_a_0["y_pred"]==1].shape[0] / df_a_0.shape[0]
print("SP_a_1:", SP_a_1)
print("SP_a_0:", SP_a_0)

# Statistical Parity Difference
SPD = SP_a_1 - SP_a_0
print("SPD:", SPD)

# Disparate Impact
DI = SP_a_0 / SP_a_1
print("DI:", DI)

# Equal Opportunity
EO_a_1 = recall_score(df_a_1[target], df_a_1['y_pred'])
EO_a_0 = recall_score(df_a_0[target], df_a_0['y_pred'])
print("EO_a_1:", EO_a_1)
print("EO_a_0:", EO_a_0)

# Equal Opportunity Difference
EOD = EO_a_1 - EO_a_0
print("EOD:", EOD)

# Accuracy per Group
ACC_a_1 = accuracy_score(df_a_1[target], df_a_1['y_pred'])
ACC_a_0 = accuracy_score(df_a_0[target], df_a_0['y_pred'])
print("ACC_a_1:", ACC_a_1)
print("ACC_a_0:", ACC_a_0)

# Accuracy per Group Difference
AGD = ACC_a_1 - ACC_a_0
print("AGD:", AGD)

# Average odds difference ------------------------------------------------------
TPR_a_1 = EO_a_1
TNR_a_1 = recall_score(df_a_1[target], df_a_1["y_pred"], pos_label = 0) 
FPR_a_1 = 1 - TNR_a_1
TPR_a_0 = EO_a_0
TNR_a_0 = recall_score(df_a_0[target], df_a_0["y_pred"], pos_label = 0)
FPR_a_0 = 1 - TNR_a_0
AvgO = ((FPR_a_1 - FPR_a_0) + (TPR_a_1 - TPR_a_0))/2
print("AvgO:", AvgO)

ACC: 0.8182115594329334
CM: [[3485  937]
 [ 730 4018]]
f1: 0.8281974647016387
AUC: 0.8171779914854769

Fairness Metrics
SP_a_1: 0.6675868788567717
SP_a_0: 0.28021248339973437
SPD: 0.38737439545703733
DI: 0.4197393511981426
EO_a_1: 0.896945551128818
EO_a_0: 0.6520854526958291
EOD: 0.24486009843298895
ACC_a_1: 0.8177979863592075
ACC_a_0: 0.8190571049136787
AGD: -0.0012591185544711392
AvgO: 0.22576938480532477


## Objective function

In [10]:
def objective_function(space):
    
    global seed, ITER, X_train, y_train, X_test, y_test
    
    ITER += 1
    
    params = {'max_depth': int(space['max_depth']), 
              'learning_rate': round(space['learning_rate'], 4),
              'n_estimators': int(space['n_estimators']),
             }
    
    print("\n------------------------------------------")
    print("\n", ITER, ":: ", params)
    
    model = LGBMClassifier(random_state=seed, n_jobs=-1, objective="binary")
    model.set_params(**params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
        
    # write ["iter", "acc", "f1-score", "auc", "recall", "cm", "params"]
    write(folder_name, [str(ITER),
           acc,
           f1,
           auc,
           recall,
           cm,
           params])

    print("ACC:", acc)
    print("CM:", cm)
    print("f1:", f1)
    print("AUC:", auc)
    
    return {'loss':-auc, 'status': STATUS_OK}

## Bayesian Optimization

In [11]:
space = {'max_depth': hp.quniform('max_depth', -1, 50, 1),
         'n_estimators': hp.quniform('n_estimators', 50, 2000, 50),
         'learning_rate': hp.uniform('learning_rate', 0.01, 0.25),
        }

header = ["iter", "acc", "f1-score", "auc", "recall", "cm", "params"]
write(folder_name, header)

ITER = 0
trials = Trials()
best = fmin(fn=objective_function,
            space=space,
            algo=tpe.suggest,
            rstate= np.random.default_rng(seed),
            max_evals=100,
            verbose=False,
            trials=trials)
print(best)


------------------------------------------

 1 ::  {'max_depth': 31, 'learning_rate': 0.2221, 'n_estimators': 1650}
ACC: 0.7945474372955289
CM: [[3450  972]
 [ 912 3836]]
f1: 0.8028463792381749
AUC: 0.7940545415680272

------------------------------------------

 2 ::  {'max_depth': 30, 'learning_rate': 0.2166, 'n_estimators': 650}
ACC: 0.8054525627044711
CM: [[3472  950]
 [ 834 3914]]
f1: 0.8143986683312526
AUC: 0.8047560885928022

------------------------------------------

 3 ::  {'max_depth': 3, 'learning_rate': 0.1582, 'n_estimators': 350}
ACC: 0.8142857142857143
CM: [[3482  940]
 [ 763 3985]]
f1: 0.8239429339398326
AUC: 0.8133636310291995

------------------------------------------

 4 ::  {'max_depth': 10, 'learning_rate': 0.1522, 'n_estimators': 850}
ACC: 0.8083969465648855
CM: [[3483  939]
 [ 818 3930]]
f1: 0.8173026931475512
AUC: 0.8076847896536312

------------------------------------------

 5 ::  {'max_depth': 4, 'learning_rate': 0.079, 'n_estimators': 1050}
ACC: 0.815049

ACC: 0.8157033805888768
CM: [[3483  939]
 [ 751 3997]]
f1: 0.825485336637753
AUC: 0.8147403920125191

------------------------------------------

 39 ::  {'max_depth': 50, 'learning_rate': 0.1106, 'n_estimators': 300}
ACC: 0.8183206106870229
CM: [[3506  916]
 [ 750 3998]]
f1: 0.8275719312771682
AUC: 0.8174463327080612

------------------------------------------

 40 ::  {'max_depth': 49, 'learning_rate': 0.1115, 'n_estimators': 300}
ACC: 0.815267175572519
CM: [[3488  934]
 [ 760 3988]]
f1: 0.8248190279214064
AUC: 0.8143579795744414

------------------------------------------

 41 ::  {'max_depth': 39, 'learning_rate': 0.1442, 'n_estimators': 550}
ACC: 0.8116684841875682
CM: [[3491  931]
 [ 796 3952]]
f1: 0.820683210466203
AUC: 0.8109061226760431

------------------------------------------

 42 ::  {'max_depth': 9, 'learning_rate': 0.1704, 'n_estimators': 350}
ACC: 0.8127589967284624
CM: [[3503  919]
 [ 798 3950]]
f1: 0.8214619943849434
AUC: 0.8120523597833761

-------------------------

ACC: 0.8149400218102508
CM: [[3491  931]
 [ 766 3982]]
f1: 0.8243453058689577
AUC: 0.8140653476128585

------------------------------------------

 77 ::  {'max_depth': 16, 'learning_rate': 0.1037, 'n_estimators': 200}
ACC: 0.8172300981461287
CM: [[3486  936]
 [ 740 4008]]
f1: 0.8270738753611225
AUC: 0.8162379875151317

------------------------------------------

 78 ::  {'max_depth': 36, 'learning_rate': 0.0404, 'n_estimators': 1200}
ACC: 0.8130861504907306
CM: [[3488  934]
 [ 780 3968]]
f1: 0.8223834196891192
AUC: 0.8122518296165644

------------------------------------------

 79 ::  {'max_depth': 48, 'learning_rate': 0.1129, 'n_estimators': 400}
ACC: 0.813413304252999
CM: [[3494  928]
 [ 783 3965]]
f1: 0.8225287833212322
AUC: 0.8126143331744433

------------------------------------------

 80 ::  {'max_depth': 32, 'learning_rate': 0.1801, 'n_estimators': 1050}
ACC: 0.802071973827699
CM: [[3475  947]
 [ 868 3880]]
f1: 0.810443864229765
AUC: 0.8015148466901916

----------------------

## Save dictionary to numpy file

In [12]:
# Ensure data types are correct
best = {
        'max_depth': int(best['max_depth']), 
        'learning_rate': best['learning_rate'],
        'n_estimators': int(best['n_estimators']),
        }
np.save('results/' + dataset + '/LGBM_hyperparameters.npy', best) 

## Best Model

In [11]:
params = np.load(folder_name + '/LGBM_hyperparameters.npy', allow_pickle='TRUE').item()

model = LGBMClassifier(random_state=seed, n_jobs=-1, objective="binary")
model.set_params(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# performance metrics
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("ACC:", acc)
print("CM:", cm)
print("f1:", f1)
print("AUC:", auc)


df_fm = pd.concat([X_test, y_test], axis=1)
df_fm['y_pred'] = y_pred

print('\nFairness Metrics')

fair_met = fairness_metrics(df_fm, protected_attribute)

# for key in fair_met.keys():
#     print(key+":", fair_met[key])

# Disparate Impact
print("DI:", fair_met["DI"])

# Statistical Parity Difference
print("SPD:", fair_met["SPD"])

# Equal Opportunity Difference
print("EOD:", fair_met["EOD"])

# Overall Accuracy Difference
print("OAD:", fair_met["OAD"])

ACC: 0.8174482006543076
CM: [[3473  949]
 [ 725 4023]]
f1: 0.8277777777777778
AUC: 0.8163476768718253

Fairness Metrics
DI: 0.42639069390885664
SPD: 0.38321349224731466
EOD: 0.23592493370026169
OAD: -0.003384642657043191
